In [30]:
# -*- coding: utf-8 -*-

'''
Simple Python 3 module for sending emails
with attachments through an SMTP server.

@author: Krystian Rosiński
'''

import requests
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm import tqdm
import csv, smtplib, ssl
import contextlib
from functools import partial
import time
from email.mime.text import MIMEText
import os
import smtplib
import getpass
import sys
#import mimetypes

from email.utils import formataddr
from email.utils import formatdate
from email.utils import COMMASPACE

from email.header import Header
from email import encoders

from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
#from email.mime.application import MIMEApplication


def send_email(passwd: str, sender_name: str, sender_addr: str, smtp: str, port: str,
               recipient_addr: list, subject: str, html: str, text: str,
               img_list: list=[], attachments: list=[],
               fn: str='last.eml', save: bool=False):


    sender_name = Header(sender_name, 'utf-8').encode()

    msg_root = MIMEMultipart('mixed')
    msg_root['Date'] = formatdate(localtime=1)
    msg_root['From'] = formataddr((sender_name, sender_addr))
    msg_root['To'] = COMMASPACE.join(recipient_addr)
    msg_root['Subject'] = Header(subject, 'utf-8')
    msg_root.preamble = 'This is a multi-part message in MIME format.'

    msg_related = MIMEMultipart('related')
    msg_root.attach(msg_related)

    msg_alternative = MIMEMultipart('alternative')
    msg_related.attach(msg_alternative)

    msg_text = MIMEText(text.encode('utf-8'), 'plain', 'utf-8')
    msg_alternative.attach(msg_text)

    msg_html = MIMEText(html.encode('utf-8'), 'html', 'utf-8')
    msg_alternative.attach(msg_html)

    for i, img in enumerate(img_list):
        with open(img, 'rb') as fp:
            msg_image = MIMEImage(fp.read())
            msg_image.add_header('Content-ID', '<image{}>'.format(i))
            msg_related.attach(msg_image)

    for attachment in attachments:
        fname = os.path.basename(attachment)

        with open(attachment, 'rb') as f:
            msg_attach = MIMEBase('application', 'octet-stream')
            msg_attach.set_payload(f.read())
            encoders.encode_base64(msg_attach)
            msg_attach.add_header('Content-Disposition', 'attachment',
                                  filename=(Header(fname, 'utf-8').encode()))
            msg_root.attach(msg_attach)

    mail_server = smtplib.SMTP(smtp, port)
    mail_server.ehlo()

    try:
        mail_server.starttls()
        mail_server.ehlo()
    except smtplib.SMTPException as e:
        print(e)

    mail_server.login(sender_addr, passwd)
    mail_server.send_message(msg_root)
    mail_server.quit()

    if save:
        with open(fn, 'w') as f:
            f.write(msg_root.as_string())

if __name__ == '__main__':
    # Usage:
    sender_name = 'Florian Eppel'
    sender_addr = 'florian.eppel@gmx.de'
    smtp = 'mail.gmx.net'
    port = '587'
    subject = 'Leaving Spirit Konzert 2023'
    text = "Konzertanfrage"
    html = """
        <p><span style="color: #ffffff;">Leaving Spirit Konzertanfrage Fall Tour 2020</span></p>
<!-- [if gte mso 9]><xml><o:OfficeDocumentSettings><o:AllowPNG/><o:PixelsPerInch>96</o:PixelsPerInch></o:OfficeDocumentSettings></xml><![endif]--><!-- [if !mso]><!--><!--<![endif]--><!-- [if !mso]><!--><!--<![endif]--><!-- [if IE]><div class="ie-browser"><![endif]-->
<table class="nl-container" style="table-layout: fixed; vertical-align: top; min-width: 320px; margin: 0 auto; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; background-color: #ffffff; width: 100%;" cellspacing="0" cellpadding="0" bgcolor="#FFFFFF">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top;" valign="top"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td align="center" style="background-color:#FFFFFF"><![endif]-->
<div style="background-color: transparent;">
<div class="block-grid two-up" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="310" style="background-color:transparent;width:310px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num6" style="max-width: 320px; min-width: 310px; display: table-cell; vertical-align: top; width: 310px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 10px 5px 10px;"><!--<![endif]-->
<div class="img-container center autowidth fullwidth" style="padding-right: 0px; padding-left: 0px;" align="center"><!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><a tabindex="-1" href="https://leavingspirit.com" target="_blank" rel="noopener"> <img class="center autowidth fullwidth" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: none; width: 100%; max-width: 290px; display: block;" title="Leaving Spirit" src="cid:image0" alt="Leaving Spirit" width="290" align="middle" border="0" /></a> <!-- [if mso]></td></tr></table><![endif]--></div>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td><td align="center" width="310" style="background-color:transparent;width:310px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num6" style="max-width: 320px; min-width: 310px; display: table-cell; vertical-align: top; width: 310px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 10px 5px 10px;"><!--<![endif]--> <!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; font-family: Tahoma, Verdana, sans-serif"><![endif]-->
<div style="color: #555555; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; line-height: 1.2; padding: 10px;">
<div style="font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; font-size: 12px; line-height: 1.2; color: #555555; mso-line-height-alt: 14px;">
<p style="font-size: 14px; line-height: 1.2; text-align: right; mso-line-height-alt: 17px; margin: 0;">Leaving Spirit</p>
<p style="font-size: 14px; line-height: 1.2; text-align: right; mso-line-height-alt: 17px; margin: 0;">Thebenweg 1</p>
<p style="font-size: 14px; line-height: 1.2; text-align: right; mso-line-height-alt: 17px; margin: 0;">97084 W&uuml;rzburg, Germany</p>
</div>
</div>
<!-- [if mso]></td></tr></table><![endif]--> <!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="620" style="background-color:transparent;width:620px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 620px; display: table-cell; vertical-align: top; width: 620px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<table class="divider" style="table-layout: fixed; vertical-align: top; border-spacing: 0px; border-collapse: collapse; min-width: 100%; width: 100%;" border="0" cellspacing="0" cellpadding="0">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding: 10px 10px 15px 10px;" valign="top">
<table class="divider_content" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px solid #222222; width: 100%;" border="0" cellspacing="0" cellpadding="0" align="center">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top">&nbsp;</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="620" style="background-color:transparent;width:620px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 620px; display: table-cell; vertical-align: top; width: 620px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]--> <!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top: 10px; padding-bottom: 30px; font-family: Tahoma, Verdana, sans-serif"><![endif]-->
<div style="color: #71777d; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; line-height: 1.5; padding: 10px 10px 30px 10px;">
<div style="font-size: 12px; line-height: 1.5; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; color: #71777d; mso-line-height-alt: 18px;">
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;"><span style="color: #000000; font-size: 14px;"><strong>Sehr geehrte Damen und Herren </strong></span>,</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;"><span style="color: #000000; font-size: 14px;"><strong>{anrede}, </strong></span>,</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">ich hatte vor einiger Zeit schonmal mit euch geschrieben bez&uuml;glich eines Konzertes meiner Blues-Rock Band <span style="text-decoration: underline;"><strong>Leaving Spirit.</strong></span> Damals hat es leider terminlich nicht geklappt, wir haben nun aber f&uuml;r 2023 noch ein paar freie Termine, die ich euch gerne anbieten w&uuml;rde. </p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Leaving Spirit spielt handgemachte Rockmusik mit Einfl&uuml;ssen aus Blues und Country, besonders wichtig ist uns dabei, dass wir als unabh&auml;ngige K&uuml;nstler alles selbst in die Hand nehmen (vom Booking bis zur Albumsproduktion), deshalb w&uuml;rde es uns besonders freuen, wenn es mit einem Konzert bei euch klappen w&uuml;rde. Folgende Termine sind f&uuml;r 2023 noch zu vergeben: 10/11 Feb, 3/4 M&auml;r, 31.3./1 Apr, 2/3 Jun, 8 Jul, 8-23 Sep, 27/28 Okt, 17/18 Nov, 8/9 Dez</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Bei R&uuml;ckfragen stehe ich euch gerne zur Verf&uuml;gung, h&ouml;rt gerne nochmal bei uns rein:</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;"><span style="text-decoration: underline;"><strong>LEAVING SPIRIT:</strong></span></p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Musikvideo "Easy Rider": <a href="https://youtu.be/6ZKZR4Pdqp4">https://youtu.be/6ZKZR4Pdqp4</a></p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Live-Session "Myth of the Delta Blues": <a href="https://youtu.be/LOETkpsoUOw">https://youtu.be/LOETkpsoUOw</a></p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Social Media Links findet Ihr unten in der Leiste!</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Pressematerial: <a href="https://www.leavingspirit.com/press.htm">https://www.leavingspirit.com/press.htm</a></p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Ich freue mich auf Eure Reaktionen und R&uuml;ckmeldungen!</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">&nbsp;</p>
<p style="font-size: 14px; line-height: 1.5; mso-line-height-alt: 21px; margin: 0;">Freundliche Gr&uuml;&szlig;e</p>
</div>
</div>
<!-- [if mso]></td></tr></table><![endif]--> <!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid two-up" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="310" style="background-color:transparent;width:310px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num6" style="max-width: 320px; min-width: 310px; display: table-cell; vertical-align: top; width: 310px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<table class="divider" style="table-layout: fixed; vertical-align: top; border-spacing: 0px; border-collapse: collapse; min-width: 100%; width: 100%;" border="0" cellspacing="0" cellpadding="0">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding: 10px;" valign="top">
<table class="divider_content" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 0px solid transparent; height: 0px; width: 100%;" border="0" cellspacing="0" cellpadding="0" align="center">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" height="0">&nbsp;</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td><td align="center" width="310" style="background-color:transparent;width:310px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num6" style="max-width: 320px; min-width: 310px; display: table-cell; vertical-align: top; width: 310px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 10px 5px 10px;"><!--<![endif]--> <!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top: 15px; padding-bottom: 0px; font-family: Tahoma, Verdana, sans-serif"><![endif]-->
<div style="color: #555555; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; line-height: 1.2; padding: 15px 0px 0px 0px;">
<div style="font-size: 12px; line-height: 1.2; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; color: #555555; mso-line-height-alt: 14px;">
<p style="font-size: 14px; line-height: 1.2; text-align: right; mso-line-height-alt: 17px; margin: 0;"><span style="color: #000000; font-size: 14px;"><strong>Florian Eppel</strong></span></p>
</div>
</div>
<!-- [if mso]></td></tr></table><![endif]--> <!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top: 0px; padding-bottom: 5px; font-family: Tahoma, Verdana, sans-serif"><![endif]-->
<div style="color: #555555; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; line-height: 1.2; padding: 0px 0px 5px 0px;">
<div style="font-size: 12px; line-height: 1.2; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; color: #555555; mso-line-height-alt: 14px;">
<p style="font-size: 12px; line-height: 1.2; text-align: right; mso-line-height-alt: 14px; margin: 0;"><span style="font-size: 12px;">Booking &amp; Management<br /></span></p>
<p style="font-size: 12px; line-height: 1.2; text-align: right; mso-line-height-alt: 14px; margin: 0;">Tel: +49 176 9678 9010</p>
<p style="font-size: 12px; line-height: 1.2; mso-line-height-alt: 14px; margin: 0;">&nbsp;</p>
</div>
</div>
<!-- [if mso]></td></tr></table><![endif]--> <!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="620" style="background-color:transparent;width:620px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 620px; display: table-cell; vertical-align: top; width: 620px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<table class="divider" style="table-layout: fixed; vertical-align: top; border-spacing: 0px; border-collapse: collapse; min-width: 100%; width: 100%;" border="0" cellspacing="0" cellpadding="0">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding: 20px 10px 20px 10px;" valign="top">
<table class="divider_content" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px dotted #CCCCCC; width: 100%;" border="0" cellspacing="0" cellpadding="0" align="center">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top">&nbsp;</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid four-up" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="155" style="background-color:transparent;width:155px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num3" style="max-width: 320px; min-width: 155px; display: table-cell; vertical-align: top; width: 155px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<div class="img-container center fixedwidth" style="padding-right: 0px; padding-left: 0px;" align="center"><!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><a tabindex="-1" href="https://instagram.com/leavingspirit" target="_blank" rel="noopener"><img class="center fixedwidth" style="text-decoration: none; -ms-interpolation-mode: bicubic; border: 0; height: auto; width: 100%; max-width: 46px; display: block;" title="Instagram" src="cid:image1" alt="instagram" width="46" align="middle" border="0" /></a> <!-- [if mso]></td></tr></table><![endif]--></div>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td><td align="center" width="155" style="background-color:transparent;width:155px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num3" style="max-width: 320px; min-width: 155px; display: table-cell; vertical-align: top; width: 155px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<div class="img-container center fixedwidth" style="padding-right: 0px; padding-left: 0px;" align="center"><!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><a tabindex="-1" href="https://facebook.com/keeprockinalive" target="_blank" rel="noopener"> <img class="center fixedwidth" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: none; width: 100%; max-width: 38px; display: block;" title="Facebook" src="cid:image2" alt="facebook" width="38" align="middle" border="0" /></a> <!-- [if mso]></td></tr></table><![endif]--></div>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td><td align="center" width="155" style="background-color:transparent;width:155px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num3" style="max-width: 320px; min-width: 155px; display: table-cell; vertical-align: top; width: 155px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<div class="img-container center fixedwidth" style="padding-right: 0px; padding-left: 0px;" align="center"><!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><a tabindex="-1" href="https://youtube.com/leavingspirit" target="_blank" rel="noopener"> <img class="center fixedwidth" style="text-decoration: none; -ms-interpolation-mode: bicubic; height: auto; border: none; width: 100%; max-width: 46px; display: block;" title="YouTube" src="cid:image3" alt="youtube" width="46" align="middle" border="0" /></a> <!-- [if mso]></td></tr></table><![endif]--></div>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td><td align="center" width="155" style="background-color:transparent;width:155px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num3" style="max-width: 320px; min-width: 155px; display: table-cell; vertical-align: top; width: 155px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<div class="img-container center fixedwidth" style="padding-right: 0px; padding-left: 0px;" align="center"><!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr style="line-height:0px"><td style="padding-right: 0px;padding-left: 0px;" align="center"><![endif]--><a tabindex="-1" href="https://leavingspirit.com" target="_blank" rel="noopener"><img class="center fixedwidth" style="text-decoration: none; -ms-interpolation-mode: bicubic; border: 0; height: auto; width: 100%; max-width: 38px; display: block;" title="Homepage" src="cid:image4" alt="homepage" width="38" align="middle" border="0" /></a> <!-- [if mso]></td></tr></table><![endif]--></div>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="620" style="background-color:transparent;width:620px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 620px; display: table-cell; vertical-align: top; width: 620px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]-->
<table class="divider" style="table-layout: fixed; vertical-align: top; border-spacing: 0px; border-collapse: collapse; min-width: 100%; width: 100%;" border="0" cellspacing="0" cellpadding="0">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td class="divider_inner" style="word-break: break-word; vertical-align: top; min-width: 100%; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%; padding: 20px 10px 20px 10px;" valign="top">
<table class="divider_content" style="table-layout: fixed; vertical-align: top; border-spacing: 0; border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; border-top: 1px dotted #CCCCCC; height: 0px; width: 100%;" border="0" cellspacing="0" cellpadding="0" align="center">
<tbody>
<tr style="vertical-align: top;" valign="top">
<td style="word-break: break-word; vertical-align: top; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;" valign="top" height="0">&nbsp;</td>
</tr>
</tbody>
</table>
</td>
</tr>
</tbody>
</table>
<!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<div style="background-color: transparent;">
<div class="block-grid" style="margin: 0 auto; min-width: 320px; max-width: 620px; overflow-wrap: break-word; word-wrap: break-word; word-break: break-word; background-color: transparent;">
<div style="border-collapse: collapse; display: table; width: 100%; background-color: transparent;"><!-- [if (mso)|(IE)]><table width="100%" cellpadding="0" cellspacing="0" border="0" style="background-color:transparent;"><tr><td align="center"><table cellpadding="0" cellspacing="0" border="0" style="width:620px"><tr class="layout-full-width" style="background-color:transparent"><![endif]--> <!-- [if (mso)|(IE)]><td align="center" width="620" style="background-color:transparent;width:620px; border-top: 0px solid transparent; border-left: 0px solid transparent; border-bottom: 0px solid transparent; border-right: 0px solid transparent;" valign="top"><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 0px; padding-left: 0px; padding-top:5px; padding-bottom:5px;"><![endif]-->
<div class="col num12" style="min-width: 320px; max-width: 620px; display: table-cell; vertical-align: top; width: 620px;">
<div style="width: 100% !important;"><!-- [if (!mso)&(!IE)]><!-->
<div style="border: 0px solid transparent; padding: 5px 0px 5px 0px;"><!--<![endif]--> <!-- [if mso]><table width="100%" cellpadding="0" cellspacing="0" border="0"><tr><td style="padding-right: 10px; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; font-family: Tahoma, Verdana, sans-serif"><![endif]-->
<div style="color: #555555; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; line-height: 1.2; padding: 10px;">
<div style="font-size: 12px; line-height: 1.2; font-family: 'Lato', Tahoma, Verdana, Segoe, sans-serif; color: #555555; mso-line-height-alt: 14px;">
<p style="font-size: 12px; line-height: 1.2; mso-line-height-alt: 14px; margin: 0;"><span style="font-size: 12px;">Copyright &copy; 2023 Leaving Spirit.&nbsp;All rights reserved.&nbsp;</span></p>
<p style="font-size: 14px; line-height: 1.2; mso-line-height-alt: 17px; margin: 0;">&nbsp;</p>
</div>
</div>
<!-- [if mso]></td></tr></table><![endif]--> <!-- [if (!mso)&(!IE)]><!--></div>
<!--<![endif]--></div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--> <!-- [if (mso)|(IE)]></td></tr></table></td></tr></table><![endif]--></div>
</div>
</div>
<!-- [if (mso)|(IE)]></td></tr></table><![endif]--></td>
</tr>
</tbody>
</table>
<!-- [if (IE)]></div><![endif]-->
        """  # template

    img_list = ["logo.png","instagram.jpg","facebook.png","youtube.png","home.png"]  # -> image0, image1, image2, ...
    attachments = []
    
    passwd = getpass.getpass('Password: ')
    

    orig_stdout = sys.stdout
    output_file = open('out.csv', 'w')
    sys.stdout = output_file

    
    #read file with venues
    with open('venues.csv') as f:
        lines = [line.rstrip('\n') for line in f]
        
        for i in tqdm(range(len(lines))):
            line=lines[i]
            linepart=line.split(",")
            name=linepart[0]
            recipient_addr=linepart[5].split(";")
            sent=linepart[1]
            stop=linepart[2]
            
            #recipient_addr=["florian.eppel@gmx.de"]
            
            if len(name)>0 and len(sent)<1 and not len(stop)>2 and i in range(320,350):
                anrede="Liebes " + name + " Team"
                message_body=html.format(anrede=anrede)
                message_body=message_body.format(text)
                
                try:
                    send_email(passwd, sender_name, sender_addr, smtp, port,
                               recipient_addr, subject, message_body, text,
                               img_list, attachments, fn='my.eml', save=True)
                    print(name+","+"yes"+","+linepart[2]+","+linepart[3]+","+linepart[4]+","+linepart[5])
                except:
                    print(name+","+"ERROR"+","+linepart[2]+","+linepart[3]+","+linepart[4]+","+linepart[5])
            else:
                print(name+","+linepart[1]+","+linepart[2]+","+linepart[3]+","+linepart[4]+","+linepart[5])
                
    sys.stdout=orig_stdout
    output_file.close()
    print("Emails sent!")
    print("Last mail was sent to " + recipient_addr[0])
    os.system("mv out.csv venues.csv")

Password: ········


100%|█████████████████████████████████████████| 981/981 [00:28<00:00, 34.08it/s]


Beatfabrik,,,Heidelberg,,
